In [15]:
# Importing (using/include) packages and files needed for the code to run 
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics 
include("/Users/christianaguirre/GitHub/gerya-julia/Grid.jl")
include("/Users/christianaguirre/GitHub/gerya-julia/Markers.jl")
include("/Users/christianaguirre/GitHub/gerya-julia/Stokes.jl")
include("/Users/christianaguirre/GitHub/gerya-julia/Temperature.jl")

# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("/Users/christianaguirre/GitHub/gerya-julia/Visualization.jl")

plots (generic function with 1 method)

In [20]:
struct Materials
    # 1 - subsurface global ocean
    # 2 - icy shell
    # 3 - sticky air
    alpha::Vector{Float64}
    rho0::Vector{Float64}
    Hr::Vector{Float64}
    Cp::Vector{Float64}
    kThermal::Vector{Float64}
    eta::Vector{Float64}
    function Materials()
         new([0.0,0.0,0.0],[1000.0,920.0,1.0],[0.0,0.0,0.0],[4180.0,2100,1.0],[0.5610,2.22,0.024],[1e12,eta_i,1e17])
    end    
end

# Creating a function to assign viscosity if eta is not constant
function compute_variable_eta(marker::Markers,materials::Materials)
    eta_i = nothing
    t_icy = nothing 
    Q = 40000 # Activation Enegry
    R_cont = 8.314 # Gas Constant 
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        if my < (0.1*90000.0*sin(mx(2*pi/90000.0))+0.5*90000.0)
            t_icy[i] = 100.0+((273.0-100.0)/((0.1*90000.0*sin(mx*(2*pi/90000.0))+0.5*90000.0)-30000.0))*(my-30000.0)
            eta_i[i] = (1e15)*exp((Q*(273.0-t_icy[i]))/(R_cont*(273.0*t_icy[i])))
        end
    end 
    return t_icy,eta_i
end 

function update_marker_density!(markers::Markers,materials::Materials)
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark                
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
    end
end

function initial_conditions!(markers::Markers,materials::Materials)
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        if my > (0.1*90000.0*sin(mx(2*pi/90000.0))+0.5*90000.0)
             # subsurface global ocean
             markers.integers[material,i] = 1
             markers.scalars[T,i] = 273.0
             markers.scalars[eta,i] = materials.eta[1]
             markers.scalars[alpha,i] = materials.alpha[1]            
        else
             # icy shell
             markers.integers[material,i] = 2
             markers.scalars[T,i] = t_icy[i]
             markers.scalars[eta,i] = materials.eta[2]
             markers.scalars[alpha,i] = materials.alpha[2]            
        end
        if my < (90000.0/3)
            # sticky air
            markers.integers[material,i] = 3
            markers.scalars[T,i] = 100.0            
            markers.scalars[eta,i] = materials.eta[3]
            markers.scalars[alpha,i] = materials.alpha[3]            
        end
        update_marker_density!(markers,materials)
    end
end

LoadError: UndefVarError: marker not defined

LoadError: type #grid has no field nx

LoadError: UndefVarError: eta not defined